In [2]:
import re
from calendar import monthrange
from os import scandir
from typing import Tuple
from PyPDF2 import PdfReader

import pandas as pd

In [6]:
MONTHS = {
    1: "Jan",
    2: "Feb",
    3: "Mar",
    4: "Apr",
    5: "May",
    6: "Jun",
    7: "Jul",
    8: "Aug",
    9: "Sep",
    10: "Oct",
    11: "Nov",
    12: "Dec"
}

In [7]:
def get_statement_transactions(file_name: str, transactions_regex: re.Pattern) -> list[str]:
    PAGE_REGEX = re.compile("INTEREST CHARGE ON PURCHASES \$ -?\d?,?\d+\.\d{2}", re.MULTILINE)
    text = ""
    year = ""
    month = ""
    with open(file_name, "rb") as file:
        year = file.name.split("-")[0][-4:]
        month = file.name.split("-")[1][:2]

        reader = PdfReader(file)
        for i, page in enumerate(reader.pages[2:], start=1):
            print(f"{year}, {month} Page: {i}")
            text += page.extract_text()

    matches = []
    for i, match in enumerate(re.finditer(transactions_regex, text), start=1):
        print(f"Match: {1}")
        if match[3]:
            day = str(monthrange(int(year), int(month))[1])
            month = MONTHS[int(month)]
            matches.append(" ".join([year, month, day, match[3]]))
        else:
            matches.append(" ".join([year, match[1], match[2]]))

    return matches

In [8]:
def get_all_transactions(folder_path: str, transactions_regex: re.Pattern) -> list[str]:
    raw_data = []
    for statement in scandir(folder_path):
        raw_data += get_statement_transactions(statement, transactions_regex)

    return raw_data

In [ ]:
OLD_PDFS = r"\data\external\Discover\Before_Change\"
OLD_REGEX = re.compile(r"(\w{3} \d{1,2}) \w{3} \d{1,2} (.* \$?-?\d?,?\d+\.\d{2})|(INTEREST CHARGE ON PURCHASES \$ -?\d?,?\d+\.\d{2})", re.MULTILINE)

NEW_PDFS = r"..\raw_data\Discover\After_Change"
NEW_REGEX = re.compile(r"\d{1,2}/\d{2}.*?\n?.*-?\$\d+\.\d{2}", re.MULTILINE)

In [12]:
get_all_transactions(OLD_PDFS, OLD_REGEX)

2015, 10 Page: 1
2015, 10 Page: 2
2015, 10 Page: 3
2015, 10 Page: 4
Match: 1
Match: 1
Match: 1
Match: 1
2015, 11 Page: 1
2015, 11 Page: 2
2015, 11 Page: 3
2015, 11 Page: 4
Match: 1
Match: 1
2015, 12 Page: 1
2015, 12 Page: 2
2015, 12 Page: 3
2015, 12 Page: 4
Match: 1
2016, 01 Page: 1
2016, 01 Page: 2
2016, 01 Page: 3
2016, 01 Page: 4
Match: 1
2016, 02 Page: 1
2016, 02 Page: 2
2016, 02 Page: 3
2016, 02 Page: 4
Match: 1
2016, 03 Page: 1
2016, 03 Page: 2
2016, 03 Page: 3
2016, 03 Page: 4
Match: 1
Match: 1
2016, 04 Page: 1
2016, 04 Page: 2
2016, 04 Page: 3
2016, 04 Page: 4
Match: 1
Match: 1
Match: 1
2016, 05 Page: 1
2016, 05 Page: 2
2016, 05 Page: 3
2016, 05 Page: 4
Match: 1
Match: 1
2016, 06 Page: 1
2016, 06 Page: 2
2016, 06 Page: 3
2016, 06 Page: 4
Match: 1
Match: 1
2016, 07 Page: 1
2016, 07 Page: 2
2016, 07 Page: 3
2016, 07 Page: 4
Match: 1
Match: 1
Match: 1
Match: 1
2016, 08 Page: 1
2016, 08 Page: 2
2016, 08 Page: 3
2016, 08 Page: 4
Match: 1
Match: 1
2016, 09 Page: 1
2016, 09 Page: 2
20

['2015 Oct 3 INTERNET PAYMENT - THANK YOU $ -265.00',
 '2015 Sep 22 FGT*EDCVEGAS 888-512-7469 TX $ 264.49',
 '2015 Sep 28 EXPEDIA*1117965480211 BELLEVUE WA 1,901.89',
 '2015 Sep 24 THE HOME DEPOT #0969 FORKED RIVER NJ $ 28.42',
 '2015 Nov 14 INTERNET PAYMENT - THANK YOU $ -150.00',
 '2015 Nov 2 REWARD STATEMENT CREDIT $ -23.08',
 '2015 Dec 19 INTERNET PAYMENT - THANK YOU $ -60.00',
 '2016 Jan 19 INTERNET PAYMENT - THANK YOU $ -100.00',
 '2016 Feb 20 INTERNET PAYMENT - THANK YOU $ -150.00',
 '2016 Mar 15 INTERNET PAYMENT - THANK YOU $ -150.00',
 '2016 Mar 23 TCKTWEB*STEVEAOKIEDCWE 800-965-4827 CA $ 61.87',
 '2016 Apr 19 INTERNET PAYMENT - THANK YOU $ -174.38',
 '2016 Apr 19 REWARD STATEMENT CREDIT $ -0.62',
 '2016 Apr 7 BUDGET.COM PREPAY 800-621-2844 NJ $ 269.56',
 '2016 May 20 INTERNET PAYMENT - THANK YOU $ -172.30',
 '2016 May 20 CASHBACK BONUS REDEMPTION PYMT/STMT CRDT $ -2.70',
 '2016 Jun 15 INTERNET PAYMENT - THANK YOU $ -175.00',
 '2016 Jun 21 BUDGET CAR AND TRUCK REN LAS VEGAS NV

In [ ]:
fn = r"..\raw_data\Discover\Before_Change\2020-01.pdf"
text = ""
year = ""
month = ""
with open(fn, "rb") as file:
        year = file.name.split("-")[0][-4:]
        month = file.name.split("-")[1][:2]
        reader = PdfReader(file)
        for page in reader.pages[2:]:
            text += page.extract_text()

for match in re.finditer(OLD_REGEX, text):
    if match[3]:
        day = str(monthrange(int(year), int(month))[1])
        month = MONTHS[int(month)]
        print(" ".join([year, month, day, match[3]]))
    else:
        print(" ".join([year, match[1], match[2]]))

In [ ]:
# clean old PDF data and append to master list
clean_data = []
old_lines = scrape_pdfs(OLD_DISCOVER_PDFS, OLD_DISCOVER_REGEX)

for line in old_lines:
    split_line = line.split()
    split_date = split_line[0:3]
    date = ['/'.join([months_dict[split_date[1]], split_date[2], split_date[0]])]
    merchant = [' '.join(x for x in split_line[5:-2])]
    amount = [split_line[-1:][0].replace(',', '')]
    clean_data.append(date + ['Discover'] + amount + merchant)

In [ ]:
# clean new PDF data and append to master list

new_lines = [line.replace('\n', ' ') if '\n' in line else line for line in new_lines] # replace newline characters
new_lines = [line[:line.index('PREVIOUS')] if 'PREVIOUS' in line else line for line in new_lines] # delete PREVIOUS in lines

for line in new_lines:
    split_line = line.split()
    date = ['/'.join([split_line[1].replace('0', ''), split_line[0]]) if split_line[1].startswith('0') else '/'.join([split_line[1], split_line[0]])]
    merchant = [' '.join(x for x in split_line[2:-1])]
    amount = [split_line[-1].replace('$', '')]
    clean_data.append(date + ['Discover'] + amount + merchant)


In [ ]:
data_types = {
    'Account': object,
    'Amount': float,
    'Recipient': object,
    'Category': object,
    'SubCategory': object,
    'Project': object,
    'Note': object
}

In [ ]:
df = pd.DataFrame(clean_data, columns=['Date', 'Account', 'Amount', 'Recipient'])
df['Category'] = np.NaN
df['SubCategory'] = np.NaN
df['Project'] = np.NaN
df['Note'] = np.NaN

df = df.astype(data_types)
df['Amount'].apply(lambda x: x * -1).sum()
df.set_index('Date', inplace=True)
df.index = pd.to_datetime(df.index, format="%m/%d/%Y", exact=True)
df.sort_index(ascending=True, inplace=True)
# df.to_excel('discover_transactions.xlsx')

In [ ]:
df.head(20)